In [1]:
#https://www.kaggle.com/code/debarshichanda/pytorch-feedback-deberta-v3-baseline
#https://github.com/renebidart/text-classification-benchmark/blob/master/run_classifier.py
#https://www.kaggle.com/code/yasufuminakama/fb3-deberta-v3-base-baseline-inference
!pip -q install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as opt
from transformers import AutoTokenizer, AutoModel, AutoConfig
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, auc, roc_curve
from copy import copy, deepcopy
import zipfile
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # specify which GPU(s) to be used
#torch.backends.cudnn.benchmark = True'''

Mounted at /content/drive


In [3]:
zf = zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/text classification/imdb 50k/archive.zip')
df = pd.read_csv(zf.open('IMDB Dataset.csv'))
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
seq_len = [len(sent) for sent in df['review']]
fig = go.Figure()
#fig.add_trace(go.Scatter(x=np.arange(len(df)), y=seq_len, mode='markers', name='Seq len'))
fig.add_trace(go.Scatter(x=np.arange(len(df)), y=[np.mean(seq_len)]*len(seq_len), mode='lines', name='Avg seq len'))
fig.add_trace(go.Scatter(x=np.arange(len(df)), y=[np.median(seq_len)]*len(seq_len), mode='lines', name='Med seq len'))
fig.show()

In [4]:
'''example_text = ['I will watch Memento tonight']
bert_input = tokenizer(example_text,padding='max_length', max_length = 10, truncation=True, return_tensors="pt")
tokenizer.decode(bert_input.input_ids[1])'''
text_column, out_column = 'review', 'sentiment'
labels = dict(zip(df[out_column].unique(), range(df[out_column].nunique())))
df.replace({out_column: labels}, inplace=True)
df_train, df_valid, df_test = np.split(df.sample(frac=1, random_state=42),  [int(.8*len(df)), int(.9*len(df))])
df_train.shape, df_valid.shape, df_test.shape

((40000, 2), (5000, 2), (5000, 2))

In [5]:
df_train

,review,sentiment
33553,I really liked this Summerslam due to the look...,0
9427,Not many television shows appeal to quite as m...,0
199,The film quickly gets to a major chase scene w...,1
12447,Jane Austen would definitely approve of this o...,0
39489,Expectations were somewhat high for me when I ...,1
...,...,...
1559,"This kind of ""inspirational"" saccharine is eno...",1
13313,"When people nowadays hear of a 1940s drama, th...",0
13528,This is a low budget Roger Corman horror/creat...,1
25017,"First off, let it be known that I came into th...",0


In [6]:
model_path = 'facebook/data2vec-text-base'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model_config = AutoConfig.from_pretrained(model_path)
data2vec = AutoModel.from_pretrained(model_path)
model_config, data2vec

Some weights of the model checkpoint at facebook/data2vec-text-base were not used when initializing Data2VecTextModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing Data2VecTextModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecTextModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(Data2VecTextConfig {
   "_name_or_path": "facebook/data2vec-text-base",
   "architectures": [
     "Data2VecTextModel"
   ],
   "attention_probs_dropout_prob": 0.1,
   "bos_token_id": 0,
   "classifier_dropout": null,
   "eos_token_id": 2,
   "hidden_act": "gelu",
   "hidden_dropout_prob": 0.1,
   "hidden_size": 768,
   "initializer_range": 0.02,
   "intermediate_size": 3072,
   "layer_norm_eps": 1e-05,
   "mask_token_id": 50264,
   "max_position_embeddings": 514,
   "model_type": "data2vec-text",
   "num_attention_heads": 12,
   "num_hidden_layers": 12,
   "pad_token_id": 1,
   "position_embedding_type": "absolute",
   "tokenizer_class": "RobertaTokenizer",
   "torch_dtype": "float32",
   "transformers_version": "4.28.1",
   "type_vocab_size": 1,
   "use_cache": true,
   "vocab_size": 50265
 },
 Data2VecTextModel(
   (embeddings): Data2VecTextForTextEmbeddings(
     (word_embeddings): Embedding(50265, 768, padding_idx=1)
     (position_embeddings): Embedding(514, 768, padding_idx=1)


In [ ]:
len(df[text_column].values[26582]), df[text_column].values[26582]

(8754,
 'Some have praised -Atlantis:-The Lost Empire- as a Disney adventure for adults. I don\'t think so--at least not for thinking adults.<br /><br />This script suggests a beginning as a live-action movie, that struck someone as the type of crap you cannot sell to adults anymore. The "crack staff" of many older adventure movies has been done well before, (think The Dirty Dozen) but -Atlantis- represents one of the worse films in that motif. The characters are weak. Even the background that each member trots out seems stock and awkward at best. An MD/Medicine Man, a tomboy mechanic whose father always wanted sons, if we have not at least seen these before, we have seen mix-and-match quirks before. The story about how one companion, Vinny played by Don Novello (Fr. Guido Sarducci), went from flower stores to demolitions totally unconvincing.<br /><br />Only the main character, Milo Thatch, a young Atlantis-obsessed academic voiced by Michael J. Fox, has any depth to him. Milo\'s sear

In [7]:
tmp = tokenizer([df[text_column].values[26582]], add_special_tokens=True, max_length=512, padding = 'max_length', truncation=True, return_tensors='pt')
tmp['input_ids'].shape,tmp

(torch.Size([1, 512]),
 {'input_ids': tensor([[    0,  6323,    33,  6425,   111, 47105, 22122, 36185,   133, 13932,
          11492,    12,    25,    10,  4672,  9733,    13,  3362,     4,    38,
            218,    75,   206,    98,  5579,   415,   513,    45,    13,  2053,
           3362, 49069,  3809,  1589, 49007,  3809, 48709,   713,  8543,  3649,
             10,  1786,    25,    10,   697,    12, 10845,  1569,     6,    14,
           2322,   951,    25,     5,  1907,     9, 28342,    47,  1395,  1331,
              7,  3362,  5988,     4,    20,    22,  8344,  2990,   813,   113,
              9,   171,  2530,  9733,  4133,    34,    57,   626,   157,   137,
              6,    36, 22111,    20, 30375,  1832,  7889,    43,    53,   111,
          47105, 22122,    12,  3372,    65,     9,     5,  3007,  3541,    11,
             14, 32847,     4,    20,  3768,    32,  3953,     4,  1648,     5,
           3618,    14,   349,   919, 14168,  1872,    66,  1302,   388,     8,
   

In [8]:
tokenizer.decode(tmp['input_ids'][0])

'<s>Some have praised -Atlantis:-The Lost Empire- as a Disney adventure for adults. I don\'t think so--at least not for thinking adults.<br /><br />This script suggests a beginning as a live-action movie, that struck someone as the type of crap you cannot sell to adults anymore. The "crack staff" of many older adventure movies has been done well before, (think The Dirty Dozen) but -Atlantis- represents one of the worse films in that motif. The characters are weak. Even the background that each member trots out seems stock and awkward at best. An MD/Medicine Man, a tomboy mechanic whose father always wanted sons, if we have not at least seen these before, we have seen mix-and-match quirks before. The story about how one companion, Vinny played by Don Novello (Fr. Guido Sarducci), went from flower stores to demolitions totally unconvincing.<br /><br />Only the main character, Milo Thatch, a young Atlantis-obsessed academic voiced by Michael J. Fox, has any depth to him. Milo\'s search fo

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SeqClsLoader(Dataset): #Sequence classification dataloader
    def __init__(self, encoded_data, labels):
        self.labels = labels
        self.encoded_data = encoded_data

    def classes(self):
        return self.labels
 
    def __len__(self):
        return len(self.encoded_data['input_ids'])
 
    def __getitem__(self, idx):
        return self.encoded_data['input_ids'][idx], self.encoded_data['attention_mask'][idx], self.labels[idx] if self.labels is not None else None

class NNModel(nn.Module):
    def __init__(self, input_shape, units=None, factors=None, activ=True, norm=False, dropout=False, slops=None):
        super().__init__()
        self.input_shape = input_shape
        self.units = units
        self.factors = factors
        self.activ, self.norm = activ, norm
        self.network = nn.ModuleList()
        if self.factors:
            self.units = np.round(self.input_shape * np.asarray(self.factors)).astype(int)
        if self.units is not None:
            self.dropout = np.zeros_like(self.units) if not dropout else dropout
            self.slops = np.full(len(self.units), 1) if slops is None else slops
            for i, j, k in zip(self.units, self.dropout, self.slops):
                if i >= 1:
                    block = self.__build_block__(input_shape, i, p=j, slop=k)
                    self.network.extend(block)
                    input_shape = i
        self.output_shape = input_shape
        self.reset_parameters()
    
    def __build_block__(self, input_shape, units, p, slop):
        block = []
        block.append(nn.Linear(input_shape, units, bias=not self.norm))
        if self.norm:
            block.append(nn.BatchNorm1d(units))
            #block.append(nn.LayerNorm(units, eps=1e-5))
        if self.activ:
            #block.append(nn.LeakyReLU())
            block.append(nn.ELU(slop))
            #block.append(nn.GELU())
        if p > 0:
            block.append(nn.Dropout(p))
        return block
 
    def forward(self, x):
        for layer in self.network:
          tmp = layer(x)
          x = tmp
        return x
 
    def reset_parameters(self):
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)
                layer.bias.data.fill_(0.1)
 

class TransSeqClassifier(nn.Module):
    def __init__(self, model, emb_dim, mlp_units, mlp_dropout, nb_class):
        super(TransSeqClassifier, self).__init__()
        self.model = model
        self.mlp = NNModel(emb_dim, units=mlp_units, factors=None, dropout=[mlp_dropout]*len(mlp_units)) if mlp_units is not None else None
        cls_units = self.mlp.output_shape if mlp_units is not None else emb_dim
        self.classifier = nn.Linear(cls_units, nb_class)

    def forward(self, input_id, mask):
        discriminator_hidden_states = self.model(input_ids= input_id, attention_mask=mask,return_dict=False)
        last_hidden_state = discriminator_hidden_states[0]
        cls_token = last_hidden_state[:, 0, :]  # take <s> token (equiv. to [CLS])
        z = self.mlp(cls_token) if self.mlp is not None else cls_token
        pred = self.classifier(z)
        return z, cls_token, pred

class BaseSeqClassifier:
    def __init__(self, model, tokenizer, max_len):
        self.model = model.to(device)
        self.losses = {'Epoch': [], 'Train': [], 'Test': [], 'BState': [], 'LState': [], 'LR': []}
        self.tokenizer, self.max_len = tokenizer, max_len
 
    def train_model(self, optim, train_loader, grad_clip, l2_reg):
          total_loss = 0
          self.model = self.model.train()
        #with autograd.detect_anomaly():
          for i, (ids, mask, Y) in enumerate(train_loader):
              ids, mask, Y = ids.to(device), mask.to(device), Y.to(device)
              #self.model.get_weight()
              optim.zero_grad()
              loss = self.loss_function(ids, mask, Y, l2_reg)
              loss.backward()
              torch.nn.utils.clip_grad_norm_(self.model.parameters(), grad_clip)
              optim.step()
              total_loss += loss.item()
          return total_loss/(i+1)
        
 
    def eval_model(self, test_loader):
        self.model = self.model.eval()
        total_loss = 0
        for i, (ids, mask, Y) in enumerate(test_loader):
            ids, mask, Y = ids.to(device), mask.to(device), Y.to(device)
            loss = self.loss_function(ids, mask, Y, l2_reg=0)
            total_loss += loss.item()
        return total_loss/(i+1)#np.abs(-100. - total_loss)
 
    def fit(self, X_train, Y_train, epoch, lr, opt_kwarg, batch_size=None,  grad_clip=100, momentum=0.9, X_test=None, Y_test=None, l2_reg=0, verbose=True, save=True):
        batch_size = len(Y_train) if batch_size is None else batch_size
        encoded_train = self.tokenizer(X_train, add_special_tokens=True, max_length=self.max_len, padding = 'max_length', truncation=True, return_tensors='pt')
        train_load = DataLoader(SeqClsLoader(encoded_train, Y_train), batch_size=batch_size, shuffle=True)  # DATALOADER obj
        if X_test is not None:
            encoded_test = self.tokenizer(X_test, add_special_tokens=True, max_length=self.max_len, padding = 'max_length', truncation=True, return_tensors='pt')
            test_load = DataLoader(SeqClsLoader(encoded_test, Y_test), batch_size=batch_size, shuffle=True)
 
        best_loss = 1e100
        optim = opt.Adam(self.model.parameters(), lr=lr)
        #optim = opt.SGD(self.model.parameters(), lr=lr, momentum=momentum, nesterov=True)

        scheduler = None
        #scheduler = opt.lr_scheduler.CyclicLR(optim, **opt_kwarg)
        #scheduler = opt.lr_scheduler.ReduceLROnPlateau(optim, **opt_kwarg)
        #scheduler = opt.lr_scheduler.MultiStepLR(optim, milestones=[28, 120], gamma=0.1)

        eval_score = ''
        for i in range(epoch):
            if verbose:
                print('##### EPOCH ' + str(i) + ' #####')
               
            train_loss = self.train_model(optim, train_load, grad_clip, l2_reg)
            self.losses['LState'] = deepcopy(self.model.state_dict())
    
            if verbose:
                print('train loss : ', train_loss)
            self.losses['Epoch'].append(i), self.losses['Train'].append(train_loss)
    
            if df_test is not None:
                valid_loss = self.eval_model(test_load)

                if verbose:
                    print('test loss : ', valid_loss)
                self.losses['Test'].append(valid_loss)
    
                if scheduler is not None:
                    scheduler.step(valid_loss)
                    self.losses['LR'].append(optim.param_groups[0]['lr'])
                    '''scheduler.step()
                    self.losses['LR'].append(scheduler.get_last_lr()[0])'''
    
                if valid_loss < best_loss:
                    self.losses['BState'] = deepcopy(self.model.state_dict())
                    best_loss = valid_loss
                    print('===========SAVE===========')


class Binaryclass(BaseSeqClassifier):#Binaryclass classification
    def __init__(self, model, tokenizer, max_len):
        super(Binaryclass, self).__init__(model, tokenizer, max_len)

    def loss_function(self, input_id, mask, Y, l2_reg):
        _, _, pred = self.model(input_id, mask)
        bce_loss = nn.BCEWithLogitsLoss()
        loss = bce_loss(pred, Y)
        return loss

    def prdict(self, X, batch_size):
        self.model.eval()
        encoded_data = self.tokenizer(X, add_special_tokens=True, max_length=self.max_len, padding = 'max_length', truncation=True, return_tensors='pt')
        data_load = DataLoader(TensorDataset(encoded_data['input_ids'],  encoded_data['attention_mask']),batch_size=batch_size)
        outputs = {'z': [], 'pooled_output': [], 'pred': []}
        for i, (ids, mask) in enumerate(data_load):
            ids, mask = ids.to(device), mask.to(device)
            z, pooled_output, pred = self.model(ids, mask)
            pred = nn.Sigmoid()(pred)
            z, pooled_output, pred = z.cpu().data.numpy(), pooled_output.cpu().data.numpy(), pred.cpu().data.numpy()
            outputs['z'].extend(z), outputs['pooled_output'].extend(pooled_output), outputs['pred'].extend(pred)
        return outputs

class Multiclass(BaseSeqClassifier):#multiclass classification
    def __init__(self, model, tokenizer, max_len):
        super(Multiclass, self).__init__(model, tokenizer, max_len)

    def loss_function(self, input_id, mask, Y, l2_reg):
        _, _, pred = self.model(input_id, mask)
        ce_loss = nn.CrossEntropyLoss()
        loss = ce_loss(pred, Y)
        return loss

    def prdict(self, X, batch_size):
        self.model.eval()
        encoded_data = self.tokenizer(X, add_special_tokens=True, max_length=self.max_len, padding = 'max_length', truncation=True, return_tensors='pt')
        data_load = DataLoader(TensorDataset(encoded_data['input_ids'],  encoded_data['attention_mask']),batch_size=batch_size)
        outputs = {'z': [], 'pooled_output': [], 'pred': []}
        for i, (ids, mask) in enumerate(data_load):
            ids, mask = ids.to(device), mask.to(device)
            z, pooled_output, pred = self.model(ids, mask)
            pred = nn.Softmax()(pred)
            z, pooled_output, pred = z.cpu().data.numpy(), pooled_output.cpu().data.numpy(), pred.cpu().data.numpy()
            outputs['z'].extend(z), outputs['pooled_output'].extend(pooled_output), outputs['pred'].extend(pred)
        return outputs

def gradient_clipper(model: nn.Module, val: float) -> nn.Module:
    def process_grad(grad):
        grad[grad != grad] = 1e-10
        return torch.clamp(grad, -val, val)
    for parameter in model.parameters():
        parameter.register_hook(lambda grad: process_grad(grad))
    
    return model

In [9]:
epoch, lr, batch_size, d, mlp_d = 50000, 1e-6, 4, 0.000001, 1e-6
#cyclic_kwarg = {'base_lr': lr, 'max_lr': 1e-2, 'step_size_up':200, 'step_size_down':200}
plateau_kwarg = {'factor':0.5, 'patience':200, 'verbose':True, 'min_lr':1e-7, 'mode':'min'}


model = TransSeqClassifier(data2vec, model_config.hidden_size, mlp_units=None, mlp_dropout=1e-6, nb_class=1)
model = gradient_clipper(model, 10)
#model.load_state_dict(best_state)
print(device)
print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
seq_bc = Binaryclass(model, tokenizer, 512)
seq_bc.fit(df_train[text_column].tolist(), df_train[out_column].values.astype(np.float32)[:,None], epoch, lr, plateau_kwarg, batch_size=batch_size, grad_clip=10, momentum=0.9,
        X_test=df_valid[text_column].tolist(), Y_test=df_valid[out_column].values.astype(np.float32)[:,None], l2_reg=0, verbose=True)

cuda
TransSeqClassifier(
  (model): Data2VecTextModel(
    (embeddings): Data2VecTextForTextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): Data2VecTextEncoder(
      (layer): ModuleList(
        (0-11): 12 x Data2VecTextLayer(
          (attention): Data2VecTextAttention(
            (self): Data2VecTextSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): Data2VecTextSelfOutput(
              (dense): Linear(in_features=768, out_feature

KeyboardInterrupt: ignored

In [ ]:
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")

NameError: ignored

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

best_state = deepcopy(seq_bc.losses['BState'])
seq_bc.model.load_state_dict(best_state)
print(np.min(seq_bc.losses['Test']))

fig = make_subplots(rows=3, cols=1)
s = 0
fig.append_trace(go.Scatter(x=seq_bc.losses['Epoch'][s:], y=seq_bc.losses['Train'][s:],mode='lines',name='Train'), row=1, col=1)
fig.append_trace(go.Scatter(x=seq_bc.losses['Epoch'][s:], y=seq_bc.losses['Test'][s:],mode='lines',name='Test'), row=2, col=1)
fig.append_trace(go.Scatter(x=seq_bc.losses['Epoch'][s:], y=seq_bc.losses['LR'][s:],mode='lines',name='LR'), row=3, col=1)
fig.update_layout(height=1000, width=1500, title_text="Stacked Subplots")
fig.show()

0.22759992176741362


In [11]:
output = seq_bc.prdict(df_valid[text_column].tolist(), 2)
pred = np.round(np.asarray(output['pred']))
auc = roc_auc_score(df_valid[out_column].values, pred)
pd.DataFrame({'AUC': auc, 'ACC': accuracy_score(df_valid[out_column].values, np.round(pred)), 
              'PRE': precision_score(df_valid[out_column].values, np.round(pred)), 'REC': recall_score(df_valid[out_column].values, np.round(pred)), 
              'F1':f1_score(df_valid[out_column].values, np.round(pred))}, index=[0])

,AUC,ACC,PRE,REC,F1
0,0.923398,0.9236,0.907189,0.945303,0.925854


In [12]:
output = seq_bc.prdict(df_test[text_column].tolist(), 2)
pred = np.round(np.asarray(output['pred']))
auc = roc_auc_score(df_test[out_column].values, pred)
pd.DataFrame({'AUC': auc, 'ACC': accuracy_score(df_test[out_column].values, np.round(pred)), 
              'PRE': precision_score(df_test[out_column].values, np.round(pred)), 'REC': recall_score(df_test[out_column].values, np.round(pred)), 
              'F1':f1_score(df_test[out_column].values, np.round(pred))}, index=[0])

,AUC,ACC,PRE,REC,F1
0,0.929168,0.929,0.909792,0.951229,0.930049


In [13]:
confusion_matrix(df_test[out_column].values, pred)

array([[2285,  234],
       [ 121, 2360]])

In [14]:
pd.concat((df_test.reset_index(drop=True), pd.DataFrame({'Pred': pred.ravel()})), axis=1)

,review,sentiment,Pred
0,This movie was borderline in crude humor....I ...,0,1.0
1,"If you have seen Friends, the writing will fee...",1,0.0
2,I was sadly disappointed by this film due to t...,1,1.0
3,Broad enough for you? Wait till you see this h...,1,1.0
4,Hitchcock was of the opinion that audiences ar...,0,1.0
...,...,...,...
4995,`Shadow Magic' recaptures the joy and amazemen...,0,0.0
4996,I found this movie to be quite enjoyable and f...,0,0.0
4997,Avoid this one! It is a terrible movie. So wha...,1,1.0
4998,This production was quite a surprise for me. I...,0,1.0
